In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import io, base64, os, json, re 
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
import cv2 
from PIL import Image
import mplfinance as mpf
import time

In [ ]:
eth = pd.read_csv('goodcleaneth.csv')
eth = eth.set_index( pd.DatetimeIndex(eth['datetime']))
eth = eth.drop(['datetime', 'MA', 'up_band', 'down_band'], axis=1)
hourly = eth[-100000:]


In [ ]:
def higher_or_lower(df):
    high = np.max(list(df['High']))
    low = np.min(list(df['Low']))
    
    return high, low

In [ ]:
def output(new, old):
    if new > old:
        return 2
    elif new < old:
        return 1
    else:
        return 0

In [ ]:

codes = {
 (0, 0): 0,
 (0, 1): 1,
 (0, 2): 2,
 (1, 0): 3,
 (1, 1): 4,
 (1, 2): 5,
 (2, 0): 6,
 (2, 1): 7,
 (2, 2): 8
}
imgs_and_label = pd.DataFrame([['random.png', 7]], columns=list(['name', 'label']))

old_high, old_low = 0,0
start = 10000
stop = 10030
area = (109, 87, 610, 589)

for i in range(500):
    if i % 20 == 0:
        time.sleep(10)
        print('Sleepin zzzzzzzz')
    df = hourly[start:stop]
    name = f'bw-minute-barchart-{i}.png'
    kwargs = dict(type='candle',mav=(2,4,6),volume=True,figratio=(10,10),figscale=1)
    mpf.plot(df,**kwargs,savefig=f'bar_imgs/{name}')
    img = Image.open(f"bar_imgs/{name}").convert('LA')

    cropped_img = img.crop(area)
    cropped_img.save(f"bar_imgs/{name}") 
    high, low = higher_or_lower(df)
    start += 30
    stop += 30
    pd_high = output(high, old_high)
    pd_low = output(low, old_low)
    label = codes.get((pd_high, pd_low))
    print(f'img {name} has high of {pd_high} and low of {pd_low}')
    
    new_row = pd.DataFrame([[name, label]], columns=list(['name','label']))
    imgs_and_label = imgs_and_label.append(new_row, ignore_index=True)
    old_high, old_low = high, low
    print(f'On number {i}')

In [ ]:
imgs_and_label['label'] = imgs_and_label['label'].shift(-1,fill_value=69)
imgs_and_label = imgs_and_label.drop([0, 500])
imgs_and_label

In [ ]:
bw_train = imgs_and_label[imgs_and_label.index % 10 != 0]
bw_val = imgs_and_label[imgs_and_label.index % 10 == 0]

In [ ]:
bw_train.to_csv('bw_train.csv', index=False)
bw_val.to_csv('bw_val.csv', index=False)